# Advent of code

### day 1

In [20]:
import pandas as pd

df = pd.read_csv("day1.csv", names=["values"])
print(df.shift(1).head())
ans = ((df - df.shift(1)) > 0).sum()[0]
print(f"The value increases {ans} times")

print("\n second \n")

print(df.rolling(3).sum().shift(-2).tail())

ans = ((df.rolling(3).sum().shift(-2) - df.rolling(3).sum().shift(-1)) > 0).sum()[0]
print(f"The value increases {ans} times")


   values
0     NaN
1   118.0
2   121.0
3   123.0
4   125.0
The value increases 1553 times

 second 

       values
1995  23858.0
1996  23872.0
1997  23881.0
1998      NaN
1999      NaN
The value increases 1597 times


# day 2

In [34]:
import pandas as pd
import numpy as np

dx = {
    "forward": 1,
    "back": -1,
    "down": 0,
    "up": 0,
}
dy = {
    "forward": 0,
    "back": 0,
    "down": -1,
    "up": 1,
}

moves = (
    pd.read_csv("day2.csv", sep=" ", names=["direction", "value"])
    .assign(dx=lambda df: df["direction"].replace(dx) * df["value"])
    .assign(dy=lambda df: df["direction"].replace(dy) * df["value"])
    [["dx", "dy"]]
)
print(moves.head(5))
print(f"The product of the final coordinates is {moves.sum().prod() * (-1)}")
moves = (
    moves
    .assign(aim=lambda df: df["dy"].cumsum() * (-1))
    .assign(dz=lambda df: (df["aim"] * df["dx"]))
    [["dx", "dz"]]
)
print(moves.head(5))
print(f"The product of the final coordinates is {moves.sum().prod()}")


   dx  dy
0   2   0
1   0  -7
2   0  -8
3   9   0
4   0  -8
The product of the final coordinates is 1670340
   dx   dz
0   2    0
1   0    0
2   0    0
3   9  135
4   0    0
The product of the final coordinates is 1954293920


# day 3

In [111]:
import pandas as pd
import numpy as np

bits = pd.read_csv("day3.csv", names=["value"], dtype=str)
p = (
    pd.DataFrame(bits["value"].apply(tuple).tolist())
    .astype(int)
    .mean()
    .round()
    .astype(int)
    .to_frame("gamma")
    .assign(epsilon=lambda df: 1 - df["gamma"])
    .astype(str)
    .sum()
    .apply(lambda x: int(x, 2))
    .prod()
)
print(f"The product of gamma and epsilon is {p}")

bits = pd.DataFrame(bits["value"].apply(tuple).tolist()).astype(int)
ix = bits.index
for col, values in bits.iteritems():
    v = values.loc[ix]
    common = int(round(v.mean() + 1e-6))
    ix = v[v == common].index
o2 = ix
ix = bits.index
for col, values in bits.iteritems():
    v = values.loc[ix]
    uncommon = int(1 - round(v.mean() + 1e-6))
    ix = v[v == uncommon].index if sum(v == uncommon) else ix
co2 = ix
q = (
    bits
    .loc[o2.union(co2), :]
    .astype(str)
    .sum(axis=1)
    .astype(int)
    .astype(str)
    .apply(lambda x: int(x, 2))
    .prod()
)
print(f"The product of o2 and co2 is {q}")

The product of gamma and epsilon is 749376
The product of o2 and c02 is 2372923


# day 4

In [81]:

import pandas as pd
import numpy as np
from io import StringIO

f = open("day4.txt", "r")
numbers = list(map(int, ''.join(f.readline()).strip().split(",")))
print("Sample:", numbers[:5])

boards = []
while True:
    board = ''.join(f.readline() for _ in range(6)).strip().replace("  ", " ").replace("\n ", "\n")
    if not board:
        break
    boards.append(pd.read_csv(StringIO(board), sep=" ", header=None))

print("Sample:\n", boards[3])

stop = False
for x in numbers:
    if stop:
        break
    for df in boards:
        df.replace(x, -1, inplace=True)
        if any((df == -1).sum(axis=0) == 5) or any((df == -1).sum(axis=1) == 5):
            print("Winning board:\n", df)
            print("x=", x)
            unmarked = df.sum().sum() + (df == -1).sum().sum()
            print("Sum of unmarked numbers:", unmarked)
            print("Product is:", x * unmarked)
            stop = True
            
stop_df = None
stop_x = None
for x in numbers:
    for i, df in enumerate(boards):
        if df is not None:
            df.replace(x, -1, inplace=True)
            if any((df == -1).sum(axis=0) == 5) or any((df == -1).sum(axis=1) == 5):
                stop_df = df
                stop_x = x
                boards[i] = None
print("Last winning board:\n", stop_df)
print("x=", stop_x)
unmarked = stop_df.sum().sum() + (stop_df == -1).sum().sum()
print("Sum of unmarked numbers:", unmarked)
print("Product is:", stop_x * unmarked)


Sample: [67, 31, 58, 8, 79]
Sample:
     0   1   2   3   4
0  87  49  12  85  88
1  67  89   7  35  70
2  37  45  93  84   9
3  80  58  54  13  22
4   8  71  48  15  39
Winning board:
     0   1   2   3   4
0  99  54  74  83  92
1  27  53  15  -1  -1
2  94  36  63  29  91
3  -1  -1  -1  -1  -1
4   9  95  23  98  33
x= 10
Sum of unmarked numbers: 1068
Product is: 10680
Last winning board:
     0   1   2   3   4
0  -1  49  71  59  90
1  -1  -1  -1  -1  -1
2  48  -1  -1  -1  -1
3  88  -1  -1  -1  -1
4  -1  -1  64  -1  -1
x= 68
Sum of unmarked numbers: 469
Product is: 31892


# day 5

In [54]:
import pandas as pd
import numpy as np
from io import StringIO
from functools import partial

f = open("day5.txt", "r")

ins = f.read().replace(" -> ", ",")
df = pd.read_csv(StringIO(ins), names=["x", "y", "r", "s"])
print("Sample:\n", df.head())
print("Number of vents: ", (df["x"] - df["r"].rename("x")).abs().sum() + (df["y"] - df["s"].rename("y")).abs().sum())
print("x:", min(df["x"].min(), df["r"].min()), max(df["x"].max(), df["r"].max()))
print("y:", min(df["y"].min(), df["s"].min()), max(df["y"].max(), df["s"].max()))

ocean = np.zeros((1000, 1000), dtype=int)

def line(arr, x, y, r, s, **kwargs):
    if x == r:
        if y < s:
            arr[x, y:(s+1)] += 1
        else:
            arr[x, s:(y+1)] += 1
    elif y == s:
        if x < r:
            arr[x:(r+1), y] += 1
        else:
            arr[r:(x+1), y] += 1        

for row in df.itertuples():
    line(arr=ocean, **row._asdict())
print("Number of locations with 2 or more vents:", (ocean >= 2).sum().sum())

ocean = np.zeros((1000, 1000), dtype=int)

def line(arr, x, y, r, s, **kwargs):
    if x == r:
        if y < s:
            arr[x, y:(s+1)] += 1
        else:
            arr[x, s:(y+1)] += 1
    elif y == s:
        if x < r:
            arr[x:(r+1), y] += 1
        else:
            arr[r:(x+1), y] += 1
    elif abs(x - r) == abs(y - s):
        for i, j in zip(range(x, r, 1 - 2 * (x > r)), range(y, s, 1 - 2 * (y > s))):
            arr[i][j] += 1
        arr[r][s] += 1

for row in df.itertuples():
    line(arr=ocean, **row._asdict())
print("Number of locations with 2 or more vents:", (ocean >= 2).sum().sum())


Sample:
      x    y    r    s
0  456  846  221  846
1  980  926   73   19
2  682  930  562  930
3  766  592  274  100
4  247  685  247   21
Number of vents:  252569
x: 10 989
y: 10 990
Number of locations with 2 or more vents: 5280
Number of locations with 2 or more vents: 16716


# day 6

In [17]:
import pandas as pd
import numpy as np
from numpy.linalg import matrix_power
from collections import Counter

f = open("day6.txt")
initial = list(map(int, f.read().strip().split(",")))
print("Sample:", initial[:5])
m = np.zeros((9, 9), dtype=int)

for i in range(8):
    m[i, i+1] = 1
m[6, 0] = 1
m[8, 0] = 1

fish = np.zeros((9,), dtype=int)

for i, j in Counter(initial).items():
    fish[i] = j
print("Number of fish after 80 days:", matrix_power(m, 80).dot(fish).sum())
print("Number of fish after 256 days:", matrix_power(m, 256).dot(fish).sum())

Sample: [3, 4, 1, 2, 1]
Number of fish after 80 days: 365131
Number of fish after 256 days: 1650309278600


# day 7

In [14]:
import pandas as pd
import numpy as np

f = open("day7.txt", "r")
crabs = list(map(int, f.read().strip().split(",")))
print("Sample:", crabs[:5])

pos = np.array(crabs, dtype=int)
a, b = pos.min(), pos.max() + 1
print("min", a, "max", b)
fuel = np.nan
ans = np.nan
for i in range(a, b):
    if not fuel < np.abs(pos - i).sum():
        ans = i
        fuel = np.abs(pos - i).sum()
print("Best horizontal:", ans)
print("Least fuel:", fuel)

fuel = np.nan
ans = np.nan
for i in range(a, b):
    if not fuel < (np.square(pos - i) + np.abs(pos - i)).sum() // 2:
        ans = i
        fuel = (np.square(pos - i) + np.abs(pos - i)).sum() // 2
print("Best horizontal:", ans)
print("Least fuel:", fuel)

Sample: [1101, 1, 29, 67, 1102]
min 0 max 1982
Best horizontal: 298
Least fuel: 356958
Best horizontal: 464
Least fuel: 105461913


# day 8

In [65]:
import pandas as pd
import numpy as np
from itertools import permutations
from functools import partial
from io import StringIO

with open("day8.txt", "r") as f:
    print("Number of 1, 4, 7, 8:", sum(len(x) in {2, 3, 4, 7} for line in f for x in line.split("|")[1].strip().split(" ")))

digits = {
    0: {0, 1, 2, 4, 5, 6},
    1: {2, 5},
    2: {0, 2, 3, 4, 6},
    3: {0, 2, 3, 5, 6},
    4: {1, 2, 3, 5},
    5: {0, 1, 3, 5, 6},
    6: {0, 1, 3, 4, 5, 6},
    7: {0, 2, 5},
    8: {0, 1, 2, 3, 4, 5, 6},
    9: {0, 1, 2, 3, 5, 6},
}
l_digits = list(zip(*sorted(digits.items())))[1]
print("Sample:\n", list(l_digits))
eligible = list(map(dict, map(partial(zip, tuple("abcdefg")), permutations(range(7)))))
print("Sample:\n", eligible[:5])
ans = 0
test = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""
# with StringIO(test) as f:
with open("day8.txt", "r") as f:
    for line in f:
        for candidate in eligible:
            stop = False
            for v in line.split("|")[0].strip().split(" "):
                if set(map(candidate.get, v)) not in l_digits:
                    stop = True
                    break
            if stop:
                continue
            else:
                ans += int(''.join(
                    map(
                        str,
                        map(
                            l_digits.index, 
                            map(
                                set, 
                                map(
                                    partial(map, candidate.get),
                                    line.split("|")[1].strip().split(" "),
                                )
                            )
                        )
                    )
                ))
                break
print("Sum:", ans)


Number of 1, 4, 7, 8: 521
Sample:
 [{0, 1, 2, 4, 5, 6}, {2, 5}, {0, 2, 3, 4, 6}, {0, 2, 3, 5, 6}, {1, 2, 3, 5}, {0, 1, 3, 5, 6}, {0, 1, 3, 4, 5, 6}, {0, 2, 5}, {0, 1, 2, 3, 4, 5, 6}, {0, 1, 2, 3, 5, 6}]
Sample:
 [{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6}, {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 6, 'g': 5}, {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 5, 'f': 4, 'g': 6}, {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 5, 'f': 6, 'g': 4}, {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 6, 'f': 4, 'g': 5}]
Sum: 1016804


# day 9

In [114]:
import numpy as np
import pandas as pd
from collections import deque
from itertools import product

cave = pd.read_csv("day9.txt", header=None)[0].str.split("", expand=True).drop([0, 101], axis=1).astype(int).set_axis(range(100), axis=1)
print("Sample:\n", cave.head())
low = (
    (cave < cave.shift(-1, axis=0).fillna(np.inf)) &
    (cave < cave.shift(-1, axis=1).fillna(np.inf)) &
    (cave < cave.shift( 1, axis=0).fillna(np.inf)) &
    (cave < cave.shift( 1, axis=1).fillna(np.inf))
)
print("Numer of lows:", int((cave[low] + 1).sum().sum()))

visited = (cave == 9)
conn_idx = (cave == 9) * (-1)
conn = []
q = []
for i, p in enumerate(product(range(100), range(100))):
    if not visited.loc[p]:
        conn.append({p})
        conn_idx.loc[p] = i
        q.append((p, p))
    else:
        conn.append({})
while len(q) > 0:
    now, prev = q.pop()
    if not visited.loc[now]:
        visited.loc[now] = True
        for di, dj in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
            if (0 <= now[0] + di < 100) and (0 <= now[1] + dj < 100) and conn_idx.loc[(now[0] + di, now[1] + dj)] != -1:
                q.append(((now[0] + di, now[1] + dj), now))
    a = conn_idx.loc[prev]
    b = conn_idx.loc[now]
    if b != a:
        big, small = (b, a) if len(conn[b]) > len(conn[a]) else (a, b)
        for p in conn[small]:
            conn_idx.loc[p] = big
        conn[big].update(conn[small])
        conn[small] == set()
l = list(sorted(map(len, conn), reverse=True))[:5]
print("Largest component sizes:", l[:5])
print("Product:", l[0] * l[1] * l[2])

Sample:
    0   1   2   3   4   5   6   7   8   9   ...  90  91  92  93  94  95  96  \
0   8   6   5   6   4   5   6   7   8   9  ...   7   7   9   7   6   5   5   
1   6   5   4   5   3   4   5   9   9   6  ...   5   6   7   9   5   4   3   
2   5   4   3   1   2   3   9   8   7   9  ...   6   7   8   9   6   5   4   
3   6   5   2   0   1   3   9   7   6   8  ...   7   8   9   9   8   7   5   
4   7   6   3   1   2   9   7   6   5   7  ...   8   9   6   9   9   8   7   

   97  98  99  
0   6   6   7  
1   1   5   6  
2   3   4   5  
3   4   5   6  
4   8   8   7  

[5 rows x 100 columns]
Numer of lows: 603
Largest component sizes: [94, 93, 90, 86, 86]
Product: 786780


# day 10

In [21]:
import numpy as np
import pandas as pd
from statistics import median
from operator import mul
from functools import reduce

d = {
    ")": "(",
    ">": "<",
    "]": "[",
    "}": "{",
}

points = {
    ")": 3,
    "]": 57,
    "}": 1197,
    ">": 25137,
}

f = open("day10.txt", "r")
ans = 0
for line in f:
    q = []
    for c in line.strip():
        if c in d and len(q) > 0 and q[-1] == d[c]:
            q.pop()
        elif c not in d:
            q.append(c)
        else:
            ans += points[c]
            break
print("Score:", ans)


points2 = {
    "(": 1,
    "[": 2,
    "{": 3,
    "<": 4,
}

f = open("day10.txt", "r")
scores = []
for line in f:
    q = []
    stop = False
    for c in line.strip():
        if c in d and len(q) > 0 and q[-1] == d[c]:
            q.pop()
        elif c not in d:
            q.append(c)
        else:
            stop = True
            break
    if not stop:
        scores.append(reduce((lambda x, y: 5 * x + y), map(points2.get, reversed(q)), 0))
print("Score:", median(scores))

Score: 294195
Score: 3490802734


# day 11

In [34]:
import numpy as np
import pandas as pd

octo = (
    pd.read_csv("day11.txt", header=None, dtype=str)
    [0]
    .str
    .split("", expand=True)
    .replace("", np.nan)
    .T
    .dropna()
    .reset_index(drop=True)
    .T
    .astype(int)
)

ans = 0
for _ in range(100):
    flash = octo.isna()
    octo += 1
    now = octo > 9
    while now.any().any():
        wave = now.shift(-1, axis=0).fillna(0) + now + now.shift(1, axis=0).fillna(0)
        wave = wave.shift(-1, axis=1).fillna(0) + wave + wave.shift(1, axis=1).fillna(0)
        octo += wave
        flash = flash | now
        octo[flash] = 0
        now = octo > 9
    ans += flash.sum().sum()
print("Number of flashes:", ans)

octo = (
    pd.read_csv("day11.txt", header=None, dtype=str)
    [0]
    .str
    .split("", expand=True)
    .replace("", np.nan)
    .T
    .dropna()
    .reset_index(drop=True)
    .T
    .astype(int)
)

i=0
flash = octo.isna()
while not flash.all().all():
    i += 1
    flash = octo.isna()
    octo += 1
    now = octo > 9
    while now.any().any():
        wave = now.shift(-1, axis=0).fillna(0) + now + now.shift(1, axis=0).fillna(0)
        wave = wave.shift(-1, axis=1).fillna(0) + wave + wave.shift(1, axis=1).fillna(0)
        octo += wave
        flash = flash | now
        octo[flash] = 0
        now = octo > 9
    ans += flash.sum().sum()
print("Step:", i)


Number of flashes: 1546
Step: 471


# day 12

In [3]:
import numpy as np
import pandas as pd
from functools import partial
from operator import add
from itertools import groupby

f = open("day12.txt", "r")
edges_in = list(map(partial(str.split, sep="-"), map(str.strip, f)))
caves = dict(map(reversed, enumerate(sorted(list(set(add(*zip(*edges_in)))), reverse=True))))
print("Caves:", caves)
print("start:", caves["start"])
print("end:", caves["end"])
edges = sorted(list(map(tuple, map(partial(map, caves.get), add(edges_in, list(map(reversed, edges_in)))))))
graph = dict(map(lambda x: (x[0], list(map(lambda y: y[1], x[1]))), groupby(edges, lambda x: x[0])))
print("Graph:", graph)

solns = []
q = [(2, ())]
while len(q) > 0:
    node, path = q.pop()
    if node < 8 and node in path:
        pass
    elif node == 6:
        solns.append(path + (node,))
    else:
        for later in graph[node]:
            q.append((later, path + (node,)))
print("Number of paths:", len(solns))

solns = []
q = [(2, ((), False))]
i = 0
while len(q) > 0:
    node, acc = q.pop()
    path, any_repeat = acc
    if node == 6:
        solns.append(path + (node,))
    elif node in path and node < 8:
        if node == 2:
            pass
        elif any_repeat:
            pass
        else:
            for later in graph[node]:
                q.append((later, (path + (node,), True)))
    else:
        for later in graph[node]:
            q.append((later, (path + (node,), any_repeat)))
print("Number of paths:", len(solns))

Caves: {'yb': 0, 'xd': 1, 'start': 2, 'qx': 3, 'pi': 4, 'jg': 5, 'end': 6, 'ej': 7, 'UF': 8, 'LD': 9, 'KN': 10, 'DM': 11}
start: 2
end: 6
Graph: {0: [1, 3, 4, 5, 8, 10], 1: [0, 6, 8, 9, 11], 2: [3, 7, 9], 3: [0, 2, 5, 7, 8, 9], 4: [0], 5: [0, 3, 7, 8, 9, 10], 6: [1, 8, 10], 7: [2, 3, 5, 9], 8: [0, 1, 3, 5, 6], 9: [1, 2, 3, 5, 7], 10: [0, 5, 6], 11: [1]}
Number of paths: 4011
Number of paths: 108035


# day 13

In [40]:
import numpy as np
import pandas as pd
from io import StringIO
from functools import reduce

f = open("day13.txt", "r")
text = f.readlines()
print("Coordinates stop in line:", text.index("\n"))

coordinates = pd.read_csv(StringIO(''.join(text[:791])), names=["x", "y"])
print("Sample coordinates:\n", coordinates.head())
folds = pd.read_csv(StringIO(''.join(text[792:])), header=None, sep=" ")[2].str.split("=", expand=True).astype({0:str, 1:int})
print("Sample folds:\n", folds)

print("x in ", coordinates["x"].min(), coordinates["x"].max())
print("y in ", coordinates["y"].min(), coordinates["y"].max())

c = coordinates.assign(x2=lambda df: 655 - (df["x"] - 655).abs())
print("before", c.groupby(["x", "y"]).count().shape, "after", c.groupby(["x2", "y"]).count().shape)

r = reduce(lambda x, y: x.assign(**{y[1]: lambda df: y[2] - (df[y[1]] - y[2]).abs()}), folds.itertuples(), coordinates).groupby(["x", "y"]).count()[[]].reset_index()
print("x in", r["x"].min(), r["x"].max())
print("y in", r["y"].min(), r["y"].max())

draw = np.resize(np.array([" "], dtype=str), (6, 39))
for _, x, y in r.itertuples():
    draw[y, x] = "#"
print('\n'.join(map(''.join, draw)))

Coordinates stop in line: 791
Sample coordinates:
      x    y
0   95  196
1  500  420
2  681  633
3   31  581
4  694  348
Sample folds:
     0    1
0   x  655
1   y  447
2   x  327
3   y  223
4   x  163
5   y  111
6   x   81
7   y   55
8   x   40
9   y   27
10  y   13
11  y    6
x in  2 1307
y in  0 894
before (791, 1) after (675, 1)
x in 0 38
y in 0 5
#  # #### #  # #  # #### ####   ## ####
#  #    # # #  #  # #    #       #    #
####   #  ##   #### ###  ###     #   # 
#  #  #   # #  #  # #    #       #  #  
#  # #    # #  #  # #    #    #  # #   
#  # #### #  # #  # #    ####  ##  ####


# day 14

In [88]:
%%time

import numpy as np
from numpy.linalg import matrix_power
import pandas as pd
import string
import json
from collections import Counter
from functools import reduce, partial
from itertools import filterfalse


f = open("day14.txt", "r")
text = f.readlines()

template = text[0].strip()
rule = dict(map(lambda x: (tuple(x[0]), x[1]), map(partial(str.split, sep=" -> "), map(str.strip, text[2:]))))

polymer = template
for _ in range(10):
    polymer = "".join(map("".join, zip(polymer[:-1], map(rule.get, zip(polymer[:-1], polymer[1:]))))) + polymer[-1]
print("Length of polymer", len(polymer), "estimate", 2**10*(len(template) - 1) + 1)
freqs = Counter(polymer).values()
print("max freq", max(freqs), "min freq", min(freqs), "max - min", max(freqs) - min(freqs))

pi = dict(map(reversed, enumerate(rule.keys())))
m = np.zeros((len(pi), len(pi)), dtype=int)
for p, c in rule.items():
    m[pi[(p[0], c)], pi[p]] += 1
    m[pi[(c, p[1])], pi[p]] += 1

alphabet = dict(map(reversed, enumerate(string.ascii_uppercase)))
w = np.zeros((len(alphabet), len(pi)), dtype=int)
for p, c in pi.items():
    w[alphabet[p[0]], c] += 1
    w[alphabet[p[1]], c] += 1

v = np.array(list(map(Counter(map(pi.get, zip(template[:-1], template[1:]))).__getitem__, range(len(pi)))), dtype=int)
count = dict(zip(
    string.ascii_uppercase,
    ((
        matrix_power(m, 40).dot(v).dot(w.T) +
        (np.arange(len(alphabet)) == alphabet[template[0]]) + 
        (np.arange(len(alphabet)) == alphabet[template[-1]])
    )//2
    ).tolist()
))

print(json.dumps(count, indent=2))
freqs = list(filter(bool, count.values()))
print("max freq", max(freqs), "min freq", min(freqs), "max - min", max(freqs) - min(freqs))


Length of polymer 19457 estimate 19457
max freq 3258 min freq 850 max - min 2408
{
  "A": 0,
  "B": 2166734856665,
  "C": 891530953320,
  "D": 0,
  "E": 0,
  "F": 1572825839452,
  "G": 0,
  "H": 1729817724725,
  "I": 0,
  "J": 0,
  "K": 2138926496435,
  "L": 0,
  "M": 0,
  "N": 1263825150498,
  "O": 2817637405111,
  "P": 3542842052072,
  "Q": 0,
  "R": 0,
  "S": 2845038574957,
  "T": 0,
  "U": 0,
  "V": 1921541874510,
  "W": 0,
  "X": 0,
  "Y": 0,
  "Z": 0
}
max freq 3542842052072 min freq 891530953320 max - min 2651311098752
CPU times: user 15.9 ms, sys: 3.25 ms, total: 19.2 ms
Wall time: 16.2 ms


# day 15

In [9]:
%%time

import numpy as np
import pandas as pd
import heapq

f = open("day15.txt")
m = np.array(list(map(list, map(str.strip, f.readlines()))), dtype=int)
a, b = m.shape
s = -np.ones((a, b), dtype=int)
print("Sample:\n", m[:5, :5])
q = [(0, (0, 0))]
i, j = None, None
while (i, j) != (a - 1, b - 1):
    v, p = heapq.heappop(q)
    i, j = p
    if s[i, j] == -1:
        s[i, j] = v
        for di, dj in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
            if (0 <= (i + di) < a) and (0 <= (j + dj) < b):
                heapq.heappush(q, (v + m[i + di, j + dj], (i + di, j + dj)))
print("Least risk:", v)


w = np.tile(m, (5, 5, 1, 1)) + (np.tile(np.arange(5), (5, 1)) + np.tile(np.arange(5), (5, 1)).T).reshape((5, 5, 1, 1))
m = (np.concatenate(w.reshape(5, 500, 100), axis=1) - 1) % 9 + 1
a, b = m.shape
aa, bb = a - 1, b - 1
s = np.ones((a, b), dtype=bool)
print("Sample:\n", m[:5, :5])
q = [(0, 0, 0)]
i, j = -1, -1
it = ((-1, 0), (0, -1), (0, 1), (1, 0))
while i != aa or j != bb:
    z += 1
    v, i, j = heapq.heappop(q)
    if s[i, j]:
        s[i, j] = False
        for di, dj in it:
            t = i + di
            u = j + dj
            if (0 <= t < a) and (0 <= u < b):
                heapq.heappush(q, (v + m[t, u], t, u))
print("Least risk:", v)


Sample:
 [[7 1 2 3 1]
 [9 7 3 8 8]
 [2 7 2 8 3]
 [3 6 9 8 2]
 [8 9 3 1 2]]
Least risk: 503
Sample:
 [[7 1 2 3 1]
 [9 7 3 8 8]
 [2 7 2 8 3]
 [3 6 9 8 2]
 [8 9 3 1 2]]
Least risk: 2853
CPU times: user 2.31 s, sys: 28.4 ms, total: 2.33 s
Wall time: 2.37 s


# day 16

In [58]:
"""
type 4 is a single packet (a number)

look for type id if type is not 4

type id 0 is total length of all subpackets given by 15 bits 

type id 1 is number of immediate subpackets given by 11 bits

00111000000000000110111101000101001010010001001000000000
VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB

version 3 bits

type 3 bits
"""

import numpy as np
import pandas as pd
from collections import namedtuple
from itertools import count
from operator import mul, lt, gt, eq
from functools import partial, reduce

"""
tuple

(version, (type, type id, the other number), (subpackets and, if literal, a number))

sum all versions... recursive tail, easy
"""

f = open("day16.txt", "r")
packet = format(int(f.read().strip(), 16), "b")

print("Length of packet:", len(packet))


"""
Parse

N, Tree, Tree, Tree, string
"""

Packet = namedtuple("Packet", ["version", "typeid", "lentypeid", "extra", "items", "rest"])

def s(packet):
    if isinstance(packet, Packet):
        return (packet.version or 0) + sum(map(s, packet.items))
    else:
        return 0

def parse(packet):
    if packet.extra == 0 or packet.rest == "":
        return packet
    else:
        version = int(packet.rest[:3], 2)
        typeid = int(packet.rest[3:6], 2)
        if typeid == 4:
            lentypeid = None
            extra = 0
            literal = []
            for a, b in zip(count(7, step=5), count(11, step=5)):
                literal.append(packet.rest[a:b])
                if packet.rest[a - 1] == "0":
                    break
            items = [int("".join(literal), 2)]
            rest = ""
            p = parse(Packet(version, typeid, lentypeid, extra, items, rest))
            packet.items.append(p)
            return parse(packet._replace(extra=packet.extra - 1, rest=packet.rest[b:]))
        else:
            lentypeid = int(packet.rest[6])
            if lentypeid == 0:
                extra = np.inf
                items = []
                rest = packet.rest[22:][:int(packet.rest[7:22], 2)]
                p = parse(Packet(version, typeid, lentypeid, extra, items, rest))
                packet.items.append(p)
                return parse(packet._replace(extra=packet.extra - 1, rest=packet.rest[22:][int(packet.rest[7:22], 2):]))
            else:
                extra = int(packet.rest[7:18], 2)
                items = []
                rest = packet.rest[18:]
                p = parse(Packet(version, typeid, lentypeid, extra, items, rest))
                packet.items.append(p._replace(rest=""))
                return parse(packet._replace(extra=packet.extra - 1, rest=p.rest))

pp = parse(Packet(None, None, None, 1, [], packet))

print("Sum of versions:", s(pp))

"""
0, sum
1, product
2, min
3, max
4, literal
5, two sub-packets, first > second is 1 else is 0
6, two sub-packets, first < second is 1 else is 0
7, two sub-packets, first == second is 1 else is 0
"""

typefcns = {
    0: sum,
    1: partial(reduce, mul),
    2: min,
    3: max,
    4: lambda x: int(*x),
    5: lambda x: int(gt(*x)),
    6: lambda x: int(lt(*x)),
    7: lambda x: int(eq(*x)),
}

def compute(packet):
    if isinstance(packet, int):
        return packet
    else:
        return typefcns[packet.typeid](map(compute, packet.items))

print("Value of packet:", compute(pp.items[0]))


Length of packet: 5528
Sum of versions: 852
Value of packet: 19348959966392


# day 17

Consider the following values:
\begin{align*}
& n: \text{ number of steps}\\
& v_x: \text{ initial speed in the direction of the $x$ axis}\\
& v_y: \text{ initial speed in the direction of the $y$ axis}
\end{align*}
The final coordinates are:
\begin{align*}
\pm (\frac{n^2 - n}{2} + |v_x|n) \ \text{or}\  \frac{n'^2 - n'}{2} \ n' \leq n , \quad \frac{-n^2 + n + 2v_y n}{2}
\end{align*}

The maximum of the $y$ coordinate is reached at $v_y$ or $v_y + 1$. Maximum is $\frac{v_y^2 + v_y}{2}$.

In [31]:
import numpy as np
import pandas as pd
from itertools import count
from math import sqrt, floor, ceil
from collections import Counter
from itertools import groupby, product

print("v_x could be (triangular):")
for n in range(30):
    if 230 <= (n**2 - n)//2 <= 283:
        print("   ", n, (n**2 - n)//2)

"""
target area: x=230..283, y=-107..-57

we'll aim to target y = -64 and see how close we get

root is 2*vy+1
"""

highest = -np.inf
for vy in range(250):
    r = (2*vy+1 + sqrt((2*vy + 1)**2 + 256))/2
    u = int(ceil(r))
    l = int(floor(r))
    if -107 <= (-u**2 + u + 2*u*vy)//2 <= -57 or -107 <= (-l**2 + l + 2*l*vy)//2 <= -57:
        highest = (vy**2 + vy)//2
print("Max y:", highest)

vys = []
vxs = []

for vy in range(-250, 250):
    for n in count(max(2*vy + 1, 0)):
        if   (-n**2 + n + 2*n*vy)//2 < -107:
            break
        elif (-n**2 + n + 2*n*vy)//2 <= -57:
            vys.append((n, vy))
        else:
            pass

for vx in range(600):
    x = 0
    for n in range(500):
        if   x > 283:
            break
        elif x >= 230:
            vxs.append((n, vx))
        else:
            pass
        x += max(vx - n, 0)
#print(dict(map(lambda x: (x[0], list(zip(*x[1]))[1]), groupby(vxs, key=lambda x:x[0]))))
#print(dict(map(lambda x: (x[0], list(zip(*x[1]))[1]), groupby(vys, key=lambda x:x[0]))))

solx = dict(map(lambda x: (x[0], tuple(zip(*x[1]))[1]), groupby(sorted(vxs), key=lambda x:x[0])))
soly = dict(map(lambda x: (x[0], tuple(zip(*x[1]))[1]), groupby(sorted(vys), key=lambda x:x[0])))

ans = []
for n in set(solx.keys()).intersection(set(soly.keys())):
    ans += list(product(solx[n], soly[n]))
#print(ans)
print("Length with different steps:", len(ans))
print("Length after removing duplicates:", len(set(ans)))

for vx, vy in set(ans):
    x = 0
    y = 0
    for n in range(500):
        if x > 283 or y < -107:
            pass
        elif x >= 230 and y <= -57:
            break
        x += max(vx - n, 0)
        y += vy - n
    if n >= 499:
        print("Something wrong with ", vx, vy)

v_x could be (triangular):
    22 231
    23 253
    24 276
Max y: 5671
Length with different steps: 4813
Length after removing duplicates: 4556


# day 18

In [89]:
import numpy as np
import pandas as pd
import json
from functools import reduce, partial
from itertools import product, starmap
from copy import deepcopy

f = open("day18.txt", "r")

def pretty(w):
    if isinstance(w, int):
        return w
    else:
        return [pretty(w[0]), pretty(w[1])]

def parse(w):
    w = json.loads(w.strip("\n "))
    for a in w:
        if isinstance(a, list):
            for b in a:
                if isinstance(b, list):
                    for c in b:
                        if isinstance(c, list):
                            for d in c:
                                if isinstance(d, list):
                                    d.append(c)
                            c.append(b)
                    b.append(a)
            a.append(w)
    w.append(None)
    return w

def explode(d):
    # explode left
    e = d
    while e[2] is not None and id(e) == id(e[2][0]):
        e = e[2]
    if e[2] is None:
        pass
    elif isinstance(e[2][0], int):
        e[2][0] += d[0]
    else:
        e = e[2][0]
        while isinstance(e[1], list):
            e = e[1]
        e[1] += d[0]
    # explode right
    e = d
    while e[2] is not None and id(e) == id(e[2][1]):
        e = e[2]
    if e[2] is None:
        pass
    elif isinstance(e[2][1], int):
        e[2][1] += d[1]
    else:
        e = e[2][1]
        while isinstance(e[0], list):
            e = e[0]
        e[0] += d[1]
    if id(d) == id(d[2][0]):
        d[2][0] = 0
    elif id(d) == id(d[2][1]):
        d[2][1] = 0
    else:
        raise ValueError("Screwed up...")

def wrap_explode(w):
    for a in w[:2]:
        if isinstance(a, list):
            for b in a[:2]:
                if isinstance(b, list):
                    for c in b[:2]:
                        if isinstance(c, list):
                            for d in c[:2]:
                                if isinstance(d, list):
                                    explode(d)
                                    return True
    return False

def wrap_split(w):
    for a in w[:2]:
        if isinstance(a, int) and a >= 10:
            if a == w[0]:
                w[0] = [a // 2, (a + 1) // 2, w]
            elif a == w[1]:
                w[1] = [a // 2, (a + 1) // 2, w]
            return True
        elif isinstance(a, list) and wrap_split(a):
            return True
    return False

def reduce_(x):
    while True:
        if wrap_explode(x):
            pass
            #print(1, pretty(x))
        elif wrap_split(x):
            pass
            #print(2, pretty(x))
        else:
            return x

def add_n_reduce(x, y):
    x = deepcopy(x)
    y = deepcopy(y)
    ans = [reduce_(x), reduce_(y), None]
    ans[0][2] = ans
    ans[1][2] = ans
    return reduce_(ans)

def magnitude(l):
    if isinstance(l, int):
        return l
    else:
        return 3 * magnitude(l[0]) + 2 * magnitude(l[1])

snails = list(map(parse, f))

print("magnitude", magnitude(reduce(add_n_reduce, snails)))

print("max magnitude", max(map(lambda x: magnitude(add_n_reduce(*x)), product(snails, snails))))


magnitude 2501
max magnitude 4935


# day 19

In [310]:
%%time

import numpy as np
import pandas as pd
from io import StringIO
from sympy.combinatorics import Permutation
from itertools import combinations, permutations, product, count
from operator import add
from functools import partial, reduce
from numpy.linalg import inv

f = open("day19.txt", "r")
dfs = [pd.read_csv(StringIO(s.strip()), skiprows=1, header=None) for s in f.read().split("\n\n")]

dfss = [pd.concat([df - v for v in df.itertuples(index=False)]).where(lambda d: (d != 0).any(axis=1)).dropna().reset_index(drop=True).astype(int).copy() for df in dfs]

overlap = []

rotations = list(map(lambda x: (*x,  Permutation(x[0]).signature() * x[1] * x[2]), product(map(list, permutations((0, 1, 2))), (-1, 1), (-1, 1))))

for i, df_i, dfd_i, j, df_j, dfd_j in map(partial(reduce, add), combinations(zip(count(0), dfs, dfss), 2)):
    stop = False
    l_df_i = list(df_i.itertuples(index=False))
    for perm, e, f, g in rotations:
        if stop:
            break
        df_jj  =  df_j[perm].set_axis([0, 1, 2], axis=1) * (e, f, g)
        dfd_jj = dfd_j[perm].set_axis([0, 1, 2], axis=1) * (e, f, g)
        if pd.concat((dfd_i, dfd_jj)).duplicated().reset_index(drop=True).sum() >= 132:
            for w in pd.concat(v - df_jj for v in l_df_i).itertuples(index=False):
                if stop:
                    break
                if pd.concat((df_i, df_jj + w)).duplicated().reset_index(drop=True).sum() >= 12:
                    overlap.append(((j, i), (np.diag((e, f, g)).dot(np.identity(3, dtype=int)[perm, :]), np.array(w))))
                    stop = True

q = overlap.copy()
q += [((x[1], x[0]), (inv(y[0]), inv(y[0]).dot(-y[1]))) for x, y in q]
iso = dict(q)
for _ in range(30):
    for x, y in q:
        for xx, yy in iso.copy().items():
            if xx[1] == x[0] and xx[0] != x[1]: 
                if (xx[0], x[1]) not in iso:
                    iso[(xx[0], x[1])] = (y[0].dot(yy[0]), y[0].dot(yy[1]) + y[1])
                else:
                    assert (iso[(xx[0], x[1])][0] == y[0].dot(yy[0])).all().all()
                    assert (iso[(xx[0], x[1])][1] == y[0].dot(yy[1]) + y[1]).all()

                    
beacons = pd.concat(pd.DataFrame(iso[(i, 0)][0]).dot(df.T).T + pd.Series(iso[(i, 0)][1]) for i, df in enumerate(dfs[1:], start=1)).astype(int)
beacons = pd.concat((dfs[0], beacons)).drop_duplicates().reset_index(drop=True)
print("Shape of scanners df after isometries and dropping duplicates and stuff:", beacons.shape)

scanners = pd.concat((pd.Series(iso[(i, 0)][1]) for i in range(1, len(dfs))), axis=1)
scanners = pd.concat((pd.Series((0, 0, 0)), scanners), axis=1).astype(int).T.reset_index(drop=True)

ans = max([(scanners - p).abs().sum(axis=1).max() for p in scanners.itertuples(index=False)])

print("Max manhattan distance:", ans)

Shape of scanners df after isometries and dropping duplicates and stuff: (330, 3)
Max manhattan distance: 9634
CPU times: user 23.4 s, sys: 228 ms, total: 23.6 s
Wall time: 23.4 s


# day 20

In [309]:
%%time

import numpy as np
import pandas as pd
from itertools import product

f = open("day20.txt", "r")

algo = list(f.readline().strip().replace(".", "0").replace("#", "1"))
f.readline()
z = np.array([list(s.strip().replace(".", "0").replace("#", "1")) for s in f], dtype=str)
m = np.concatenate((np.tile("0", (z.shape[0], 4)), z, np.tile("0", (z.shape[0], 4))), axis=1)
m = np.concatenate((np.tile("0", (4, m.shape[1])), m, np.tile("0", (4, m.shape[1]))), axis=0)

mm = np.tile("0", m.shape)
for i, j in product(range(1, m.shape[0]), range(1, m.shape[1])):
    mm[i, j] = algo[int(''.join(map(''.join, m[i - 1: i + 2, j - 1: j + 2])), 2)]

mmm = np.tile("0", m.shape)
for i, j in product(range(2, m.shape[0] - 1), range(2, m.shape[1] - 1)):
    mmm[i, j] = algo[int(''.join(map(''.join, mm[i - 1: i + 2, j - 1: j + 2])), 2)]

print("Number of lit cells: ", mmm.astype(int).sum())

m = np.concatenate((np.tile("0", (z.shape[0], 100)), z, np.tile("0", (z.shape[0], 100))), axis=1)
m = np.concatenate((np.tile("0", (100, m.shape[1])), m, np.tile("0", (100, m.shape[1]))), axis=0)
mm = m.copy()

for k in range(50):
    m = mm
    mm = np.tile("0", m.shape)
    for i, j in product(range(1 + k, m.shape[0] - k), range(1 + k, m.shape[1] - k)):
        mm[i, j] = algo[int(''.join(map(''.join, m[i - 1: i + 2, j - 1: j + 2])), 2)]

print("Number of lit cells: ", mm.astype(int).sum())

Number of lit cells:  5057
Number of lit cells:  18502
CPU times: user 19.4 s, sys: 119 ms, total: 19.5 s
Wall time: 19.7 s


# day 21

In [35]:
"""
input:
Player 1 starting position: 6
Player 2 starting position: 7
"""

import numpy as np
import pandas as pd
from itertools import cycle, product
from functools import lru_cache, partial
from collections import Counter
from operator import mul

initial = [5, 6] # subtract 1 to the indices in order to use modular arithmetic
pos = initial.copy()
points = [0, 0]
die = cycle(range(10))
print("Ignore this die roll:", next(die))
rolls = 0

for player in cycle([0, 1]):
    rolls += 3
    pos[player] += sum(next(die) for _ in range(3))
    pos[player] %= 10
    points[player] += pos[player] + 1
    if points[player] >= 1000:
        break
print("rolls", rolls, "winning player", player,"points of losing player", points[1 - player], "product", rolls * points[1 - player])

rolls = Counter(map(sum, product(range(1, 4), range(1, 4), range(1, 4))))

@lru_cache(maxsize=None)
def outcomes(pos1, pos2, points1, points2):
    if points1 >= 21:
        return [0, 0]
    elif points2 >= 21:
        return [0, 1]
    else:
        ans = [0, 0]
        for result, nrolls in rolls.items():
            pos = (pos1 + result) % 10
            points = pos + 1 + points1
            ans = list(map(lambda x: x[0] + nrolls * x[1], zip(ans, outcomes(pos2, pos, points2, points)[::-1])))
        return ans

wins_per_player = outcomes(*initial, 0, 0)
print("Wins per player", wins_per_player)
print("Number of wins of the player that wins the most", max(wins_per_player))

print("Solving it again for lols")

universe = {}
q = [(*initial, 0, 0, 0)]
while q:
    conditions = q.pop()
    pos1, pos2, points1, points2, state = conditions
    if conditions[:-1] in universe:
        pass
    elif points1 >= 21:
        universe[conditions[:-1]] = (0, 0)
    elif points2 >= 21:
        universe[conditions[:-1]] = (0, 1)
    elif state == 0:
        q.append((*conditions[:-1], 1))
        ans = [0, 0]
        for result, _ in rolls.items():
            pos = (pos1 + result) % 10
            points = pos + 1 + points1
            q.append((pos2, pos, points2, points, 0))
    elif state == 1:
        ans = (0, 0)
        for result, nrolls in rolls.items():
            pos = (pos1 + result) % 10
            points = pos + 1 + points1
            ans = tuple(map(lambda x: x[0] + nrolls * x[1], zip(ans, universe.get((pos2, pos, points2, points))[::-1])))
        universe[conditions[:-1]] = ans
    else:
        raise ValueError("Screwed up...")

print("Wins per player", universe[(*initial, 0, 0)])


Ignore this die roll: 0
rolls 1005 winning player 0 points of losing player 917 product 921585
Wins per player [911090395997650, 645891163888987]
Number of wins of the player that wins the most 911090395997650
Solve it again for lols
Wins per player (911090395997650, 645891163888987)


# day 22

In [34]:
%%time

import pandas as pd
import numpy as np
from io import StringIO

f = open("day22.txt", "r")
text = f.read().replace("on ", "1,").replace("off ", "0,").replace("x=", "").replace("y=", "").replace("z=", "").replace("..", ",")
steps = pd.read_csv(StringIO(text), names=["state", "x0", "x1", "y0", "y1", "z0", "z1"]).astype(int).astype({"state": bool})

m = np.zeros((101, 101, 101), dtype=bool)
for state, x0, x1, y0, y1, z0, z1 in steps.itertuples(index=False):
    m[
        (50 + min(51, max(-50, x0))):(50 + min(51, max(-50, x1 + 1))),
        (50 + min(51, max(-50, y0))):(50 + min(51, max(-50, y1 + 1))),
        (50 + min(51, max(-50, z0))):(50 + min(51, max(-50, z1 + 1))),
    ] = state 
print("Number of cubes on:", m.sum())

xs = sorted(list(set(steps["x0"].tolist() + (steps["x1"] + 1).tolist())))
ys = sorted(list(set(steps["y0"].tolist() + (steps["y1"] + 1).tolist())))
zs = sorted(list(set(steps["z0"].tolist() + (steps["z1"] + 1).tolist())))
n_x = len(xs)
n_y = len(ys)
n_z = len(zs)
print("# x endoints:", n_x, "# y endoints:", n_y, "# z endoints:", n_z)

dx = np.array(xs[1:], dtype=int) - np.array(xs[:-1], dtype=int)
dy = np.array(ys[1:], dtype=int) - np.array(ys[:-1], dtype=int)
dz = np.array(zs[1:], dtype=int) - np.array(zs[:-1], dtype=int)
mm = (
    np.tile(dx.reshape((-1, 1, 1)), (1, n_y - 1, n_z - 1)) *
    np.tile(dy.reshape((1, -1, 1)), (n_x - 1, 1, n_z - 1)) *
    np.tile(dz.reshape((1, 1, -1)), (n_x - 1, n_y - 1, 1))
)
mmm = np.zeros((n_x - 1, n_y - 1, n_z - 1), dtype=bool)
for state, x0, x1, y0, y1, z0, z1 in steps.itertuples(index=False):
    ix0 = xs.index(x0)
    ix1 = xs.index(x1 + 1)
    iy0 = ys.index(y0)
    iy1 = ys.index(y1 + 1)
    iz0 = zs.index(z0)
    iz1 = zs.index(z1 + 1)
    mmm[ix0:ix1, iy0:iy1, iz0:iz1] = state
print("Number of cubes on:", (mm * mmm).sum())

Number of cubes on: 615869
# x endoints: 834 # y endoints: 833 # z endoints: 834
Number of cubes on: 1323862415207825
CPU times: user 14.9 s, sys: 29.5 s, total: 44.4 s
Wall time: 55.2 s


# day 22
Solution by Exaphis (Github)

In [ ]:
%%time

from typing import Callable, Any


def run(
    part_1: Callable[[str], Any], part_2: Callable[[str], Any], input_fname: str
) -> None:
    input_data = input_fname
    try:
        with open(input_fname) as f:
            input_data = f.read()
    except (FileNotFoundError, OSError):
        print("File not found, treating as string.")

    print("[+] Part 1 output:")
    print(part_1(input_data))

    print("------------------")
    print("[+] Part 2 output:")
    print(part_2(input_data))

import sys
import os
from typing import Generator
from collections import defaultdict
from dataclasses import dataclass
from copy import copy
from pathlib import Path

local_dir = Path().absolute().as_posix()
sys.path.append(os.path.join(local_dir, ".."))


@dataclass
class Prism:
    x: int
    y: int
    z: int
    dx: int
    dy: int
    dz: int

    def volume(self):
        return self.dx * self.dy * self.dz

    def difference(self, other):
        if not self.overlap(other):
            return [self]

        return list(prism_difference(self, other))

    def overlap(self, other):
        overlap_x = (self.x + self.dx > other.x) and (self.x < other.x + other.dx)
        overlap_y = (self.y + self.dy > other.y) and (self.y < other.y + other.dy)
        overlap_z = (self.z + self.dz > other.z) and (self.z < other.z + other.dz)
        return overlap_x and overlap_y and overlap_z


def prism_difference(p1c: Prism, p2c: Prism):
    p1 = copy(p1c)
    p2 = copy(p2c)

    bottom = Prism(p1.x, p1.y, p1.z, p1.dx, p1.dy, p2.z - p1.z)
    top = Prism(p1.x, p1.y, p2.z + p2.dz, p1.dx, p1.dy, (p1.z + p1.dz) - (p2.z + p2.dz))

    new_max_z = min(p1.z + p1.dz, p2.z + p2.dz)
    p1.z = max(p1.z, p2.z)
    p1.dz = new_max_z - p1.z

    s1 = Prism(p1.x, p1.y, p1.z, p1.dx, p2.y - p1.y, p1.dz)
    s2 = Prism(p1.x, p2.y + p2.dy, p1.z, p1.dx, (p1.y + p1.dy) - (p2.y + p2.dy), p1.dz)

    new_max_y = min(p1.y + p1.dy, p2.y + p2.dy)
    p1.y = max(p1.y, p2.y)
    p1.dy = new_max_y - p1.y

    s3 = Prism(p1.x, p1.y, p1.z, p2.x - p1.x, p1.dy, p1.dz)
    s4 = Prism(p2.x + p2.dx, p1.y, p1.z, (p1.x + p1.dx) - (p2.x + p2.dx), p1.dy, p1.dz)

    outs = [top, bottom, s1, s2, s3, s4]
    for p in outs:
        if p.dx > 0 and p.dy > 0 and p.dz > 0:
            yield p


def part1(data: str):
    cubes = [[[False] * 100 for _ in range(100)] for _ in range(100)]
    for line in data.splitlines():
        state, pos = line.split(" ")
        xs, ys, zs = pos.split(",")
        xs = xs[2:]
        ys = ys[2:]
        zs = zs[2:]
        x_min, x_max = map(int, xs.split(".."))
        y_min, y_max = map(int, ys.split(".."))
        z_min, z_max = map(int, zs.split(".."))

        x_min = max(x_min, -50)
        y_min = max(y_min, -50)
        z_min = max(z_min, -50)
        x_max = min(x_max, 50)
        y_max = min(y_max, 50)
        z_max = min(z_max, 50)

        for x in range(x_min, x_max + 1):
            for y in range(y_min, y_max + 1):
                for z in range(z_min, z_max + 1):
                    cubes[x + 50][y + 50][z + 50] = state == "on"
    cnt = 0
    for x in range(100):
        for y in range(100):
            for z in range(100):
                if cubes[x][y][z]:
                    cnt += 1
    return cnt


def part2(data: str):
    prisms: list[Prism] = []
    for i, line in enumerate(data.splitlines()):
        # print(i)
        state, pos = line.split(" ")
        xs, ys, zs = pos.split(",")
        xs = xs[2:]
        ys = ys[2:]
        zs = zs[2:]

        x_min, x_max = map(int, xs.split(".."))
        y_min, y_max = map(int, ys.split(".."))
        z_min, z_max = map(int, zs.split(".."))
        x_max += 1
        y_max += 1
        z_max += 1

        p = Prism(x_min, y_min, z_min, x_max - x_min, y_max - y_min, z_max - z_min)
        if state == "on":
            to_add = [p]

            # remove existing prisms from new prism
            for p2 in prisms:
                new_to_add: list[Prism] = []
                for p1 in to_add:
                    new_to_add.extend(p1.difference(p2))

                to_add = new_to_add

            prisms.extend(to_add)
        else:
            new_prisms: list[Prism] = []
            for p1 in prisms:
                new_prisms.extend(p1.difference(p))

            prisms = new_prisms

        # prisms list should be disjoint

    cnt = 0
    for p in prisms:
        cnt += p.volume()

    return cnt


run(lambda x: None, part2, os.path.join(local_dir, "day22.txt"))

# day 22

Solution by alykzandr (reddit)

In [ ]:
%%time

import re
from typing import List, Tuple, Union, Any
from pathlib import Path

DAY = '22'


def load_data(infile_path: str) -> List[List[Union[str, Any]]]:
    data = []
    with open(infile_path, 'r', encoding='ascii') as infile:
        for line in infile:
            split_data = \
                re.match(r'(\w+) x=(-?\d+)\.\.(-?\d+),y=(-?\d+)\.\.(-?\d+),z=(-?\d+)\.\.(-?\d+)$',
                         line).groups()
            data.append([split_data[0]] + [int(i) for i in split_data[1:]])
    return data


def overlapping_box(box_a: List[int], box_b: List[int]) -> Tuple[int, ...]:
    max_x, max_y, max_z = [max(box_a[i], box_b[i]) for i in (0, 2, 4)]
    min_xp, min_yp, min_zp = [min(box_a[i], box_b[i]) for i in (1, 3, 5)]
    if min_xp - max_x >= 0 and min_yp - max_y >= 0 and min_zp - max_z >= 0:
        return max_x, min_xp, max_y,  min_yp, max_z, min_zp


def count_lit_cubes(data, verbose=True):
    lit_count = 0
    counted_zones = []
    for i, d in enumerate(reversed(data)):
        if verbose:
            # print(i, d)
        mode, box = d[0], d[1:]
        x1, x2, y1, y2, z1, z2 = box
        if mode == 'on':
            dead_cubes = []
            for overlap_box in [overlapping_box(zone, box) for zone in counted_zones]:
                if overlap_box:
                    dead_cubes.append(('on', *overlap_box))
            lit_count += (x2 - x1 + 1) * (y2 - y1 + 1) * (z2 - z1 + 1)
            lit_count -= count_lit_cubes(dead_cubes, verbose=False)
        counted_zones.append(box)
    return lit_count


def part_1(infile_path:str) -> int:
    data = []
    for row in load_data(infile_path):
        x1, x2, y1, y2, z1, z2 = row[1:]
        if x1 <= 50 and x2 >= -50 and y1 <= 50 and y2 >= -50 and z1 <= 50 and z2 >= -50:
            data.append(row)
    lit_count = count_lit_cubes(data)
    return lit_count


def part_2(infile_path: str) -> int:
    data = load_data(infile_path)
    lit_count = count_lit_cubes(data)
    return lit_count


part2_answer = part_2((Path().absolute() / "day22.txt").as_posix())
print(f'Part 2: {part2_answer}')

# day 23

In [211]:
%%time

import numpy as np
import pandas as pd
from io import StringIO
import re
from heapq import heappush, heappop, heapify

text = """
#############
#...........#
###D#C#A#B###
  #C#D#A#B#
  #########
"""

"""
Don't stop in (2, 2), (4, 2), (6, 2), (8, 2)
Once it moves in the halway go to the final position
"""

print("Using pencil, the answer is", 17400)

"""
new input
"""

text = """
#############
#...........#
###D#C#A#B###
  #D#C#B#A#
  #D#B#A#C#
  #C#D#A#B#
  #########
"""

burrow = tuple(pd.read_csv(StringIO(re.sub(r"###|(?<=\s)#|#(?=\s)| ", "", text)), sep="#", skipfooter=0, skiprows=3, index_col=None, header=None, engine="python").T.itertuples(index=False, name=None))
burrow = (".", ".") + burrow[0:1] + (".",) + burrow[1:2] + (".",) + burrow[2:3] + (".",) + burrow[3:4] + (".", ".")

fuel = {
    ".": 0,
    "A": 1,
    "B": 10,
    "C": 100,
    "D": 1000,
}
rooms = {
    "A": 2,
    "B": 4,
    "C": 6,
    "D": 8,
}

ishallway = {0, 1, 3, 5, 7, 9, 10}.__contains__
isroom = {2, 4, 6, 8}.__contains__
isempty = (".", ".", ".", ".").__eq__


def isvalid(move, position):
    i0, i1 = move
    if isroom(i0):
        return position[i1] == "." and all(map(lambda x: (x == ".") or isinstance(x, tuple), position[i1:i0:1 - 2 * (i1 > i0)]))
    else:
        return not bool(set(position[i1]) - {".", position[i0]}) and all(map(lambda x: (x == ".") or isinstance(x, tuple), position[i1:i0:1 - 2 * (i1 > i0)]))

def explore(position):
    for i0 in [0, 1, 3, 5, 7, 9, 10]:
        amphipod = position[i0]
        if amphipod != ".":
            i1 = rooms[amphipod]
            if isvalid((i0, i1), position):
                room = position[i1]
                energy = abs(i1 - i0) * fuel[amphipod] + fuel[amphipod] + sum(map(fuel.get, room))
                yield energy, (i0, i1)
    for i0 in [2, 4, 6, 8]:
        room = position[i0]
        amphipod = room[0]
        if amphipod != ".":
            for i1 in [0, 1, 3, 5, 7, 9, 10]:
                if position[i1] == "." and isvalid((i0, i1), position):
                    energy = abs(i1 - i0) * fuel[amphipod] + sum(map(fuel.get, room))
                    yield energy, (i0, i1)

def step(move, position):
    i0, i1 = move
    ll = list(position)
    if ishallway(i0):
        ll[i1] = (ll[i0],) + position[i1][:-1]
        ll[i0] = "."
    elif isroom(i0):
        ll[i1] = ll[i0][0]
        ll[i0] = position[i0][1:] + (".",)
    else:
        assert False, "Screwed up..."
    return tuple(ll)
    
positions = set()
q = [(0, burrow)]

i = 0
position = [()] * 10
while not (set(position[2]) == {"A"} and set(position[4]) == {"B"} and set(position[6]) == {"C"} and set(position[8]) == {"D"}):
    acc_energy, position = heappop(q)
    if position in positions:
        pass
    else:
        positions.add(position)
        for energy, move in explore(position):
            heappush(q, (acc_energy + energy, step(move, position)))

print("Least energy:", acc_energy)
        
        

Using pencil, the answer is 17400
Least energy: 46120
CPU times: user 2.67 s, sys: 31.8 ms, total: 2.7 s
Wall time: 2.71 s


# day 24

In [401]:
import numpy as np
import pandas as pd
from collections import defaultdict
from random import choices
from functools import reduce
from itertools import count, starmap
from operator import add, mul, eq, mod

df = pd.read_csv("day24.txt", names=["operation", "x", "y"], sep=" ").fillna(0).astype({"y":str})
alu = [t if t[2].isalpha() else t[:2] + (int(t[2]),) for t in df.itertuples(index=False, name=None)]

"""
inp
mul
add
mod do not use negative numbers.
div do not divide by zero. truncate the result (round towards 0).
eql

instructions show repetition every 18
abuse it a bit and try to make "eql x w" return 1 whenever x is less than 10
"""

def monad(instructions, nr):
    enter = iter(map(int, list(str(nr))))
    operations = {
        "inp": lambda *args: next(enter),
        "eql": lambda x, y: int(x == y),
        "add": add,
        "mul": mul,
        "mod": mod,
        "div": lambda x, y: (abs(x) // abs(y)) * int(np.sign(x * y)),
    }
    variables = defaultdict(lambda: 0)
    for op, x, y in instructions:
        if y == "w" and op == "eql":
            print(op, x, y, dict(variables))
        #print(op, x, y)
        #print(variables)
        y = y if isinstance(y, int) else variables[y]
        if op == "div":
            assert y != 0, "don't divide by zero"
        elif op == "mod":
            assert y > 0 and variables[x] >= 0, "don't use negative numbers in integer division"

        variables[x] = operations[op](variables[x], y)
    
    return variables["z"] == 0, dict(variables)
        
for _ in range(1):
    try:
        nr = reduce(lambda x, y: 10 * x + y, (9,9,7,9,9, 2,1, 2, 9, 4, 9,9,6,7) + tuple(choices(range(1, 10), k=14)))
        valid, variables = monad(alu, nr)
        if True:
            print(valid, variables, flush=True)
    except AssertionError:
        pass

print("by trial and error answer is", "".join(map(str, (9,9,7,9,9, 2,1, 2, 9, 4, 9,9,6,7))))

for _ in range(1):
    try:
        nr = reduce(lambda x, y: 10 * x + y, (3,4,1,9,8, 1,1, 1, 8, 1, 6,3,1,1) + tuple(choices(range(1, 10), k=14)))
        valid, variables = monad(alu, nr)
        if True:
            print(valid, variables, flush=True)
    except AssertionError:
        pass

print("by trial and error answer is", "".join(map(str, (3,4,1,9,8, 1,1, 1, 8, 1, 6,3,1,1))))


eql x w {'w': 9, 'x': 14, 'z': 0}
eql x w {'w': 9, 'x': 34, 'z': 21, 'y': 21}
eql x w {'w': 7, 'x': 27, 'z': 561, 'y': 15}
eql x w {'w': 9, 'x': 25, 'z': 14597, 'y': 11}
eql x w {'w': 9, 'x': 27, 'z': 379536, 'y': 14}
eql x w {'w': 2, 'x': 2, 'z': 379536, 'y': 9}
eql x w {'w': 1, 'x': 1, 'z': 14597, 'y': 0}
eql x w {'w': 2, 'x': 21, 'z': 14597, 'y': 0}
eql x w {'w': 9, 'x': 9, 'z': 14597, 'y': 16}
eql x w {'w': 4, 'x': 22, 'z': 14597, 'y': 0}
eql x w {'w': 9, 'x': 9, 'z': 14597, 'y': 18}
eql x w {'w': 9, 'x': 9, 'z': 561, 'y': 0}
eql x w {'w': 6, 'x': 6, 'z': 21, 'y': 0}
eql x w {'w': 7, 'x': 7, 'z': 0, 'y': 0}
True {'w': 7, 'x': 0, 'z': 0, 'y': 0}
by trial and error answer is 99799212949967
eql x w {'w': 3, 'x': 14, 'z': 0}
eql x w {'w': 4, 'x': 28, 'z': 15, 'y': 15}
eql x w {'w': 1, 'x': 22, 'z': 400, 'y': 10}
eql x w {'w': 9, 'x': 19, 'z': 10405, 'y': 5}
eql x w {'w': 8, 'x': 27, 'z': 270544, 'y': 14}
eql x w {'w': 1, 'x': 1, 'z': 270544, 'y': 8}
eql x w {'w': 1, 'x': 1, 'z': 10405,

In [36]:
import numpy as np
import pandas as pd

f = open("day25.txt", "r")

m = np.array(list(map(list, map(str.strip, f.readlines()))), dtype=str)
w = m.copy()
move_right, move_down = np.array([1]), np.array([1])
ans = 0
while np.any(move_right | move_down):
    move_right = (w == ">") & (np.roll(w, -1, axis=1) == ".")
    w[move_right] = "."
    w[np.roll(move_right, 1, axis=1)] = ">"
    move_down  = (w == "v") & (np.roll(w, -1, axis=0) == ".")
    w[move_down] = "."
    w[np.roll(move_down,  1, axis=0)] = "v"
    ans += 1
    
print("sea cucumbers stop on step:", ans)

sea cucumbers stop on step: 417
